# test qwen2

In [1]:
# 增加norm和rope的一致性
import os
import sys
import torch
import numpy  as np
from deepspeed.profiling.flops_profiler import get_model_profile  # type: ignore
current_dir = os.getcwd()
print(current_dir)
sys.path.append(os.path.join(current_dir, "steel_modify_from_qwen_1_5"))
from transformers import AutoConfig
from steel_modify_from_qwen_1_5.modeling_steel import SteelForCausalLM
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)
# config.use_cuda_rmsnorm = True
# config.mlp_type = "senet"
# config.FFN_type = "softmoe_v3"
# config.intermediate_size = 1792
# config.hidden_size = 1792
# config.num_attention_heads = 32
# config.num_hidden_layers = 18
# config.mlp_div_ratio = 4
# ['eager', 'flash_attention_2', 'sdpa']
# config._attn_implementation = "flash_attention_2"
model = SteelForCausalLM(config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
memory_allocated = torch.cuda.memory_allocated(device)
print("模型在显存中的占用大小：", memory_allocated, "字节")
for name, param in model.named_parameters():
    print(f"参数名称: {name}，数据类型: {param.dtype}")
    break
# print(model)

[2024-05-18 14:53:48,068] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
/home/calfa100/gqs/Steel-LLM/pretrain_modify_from_TinyLlama/model
zhanshijin: surrport flash_attn_2
zhanshijin: if flash attn surrport window:True


mlp_class: SteelSENet
FFN: SteelSoftMoeV3
zhanshijin: now use _attn_implementation is sdpa, you can choose from dict_keys(['eager', 'flash_attention_2', 'sdpa'])


zhanshijin: use torch rmsnorm
模型在显存中的占用大小： 4765387776 字节
参数名称: model.embed_tokens.weight，数据类型: torch.float32


In [2]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1792)
    (layers): ModuleList(
      (0-17): 18 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1792, out_features=1792, bias=True)
          (k_proj): Linear(in_features=1792, out_features=1792, bias=True)
          (v_proj): Linear(in_features=1792, out_features=1792, bias=True)
          (o_proj): Linear(in_features=1792, out_features=1792, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): SteelSoftMoEV3(
          (experts): ModuleList(
            (0-5): 6 x SteelSENet(
              (gate_up_proj): Linear(in_features=1792, out_features=448, bias=False)
              (up_proj): Linear(in_features=1792, out_features=448, bias=False)
              (gate_down_proj): Linear(in_features=448, out_features=1792, bias=False)
              (down_proj): Linear(in_features=448, out_features=1792, bias=False)
              (act_

In [2]:

# RuntimeError: FlashAttention only support fp16 and bf16 data type
seed = 666
torch.manual_seed(seed)  # 设置随机种子
torch.cuda.manual_seed(seed)  # 设置CUDA的随机种子
torch.cuda.manual_seed_all(seed)  # 如果使用多个GPU，设置所有GPU的随机种子
np.random.seed(seed)
fake_input = torch.randint(1, 10000, (1, 256),dtype=torch.long, device=model.device)
input_dict = {"input_ids": fake_input, "labels": fake_input.clone()}
# model = model.to(torch.float32)
# flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
# print("FLOPs:", flops)
# print("MACs:", macs)
# print("Params:", params)
# output = model(fake_input)
# print(output.logits[:,:,:10])
# print("*"*20)
#===========================================================
model = model.to(torch.bfloat16)
flops, macs, params = get_model_profile(model, kwargs=input_dict, print_profile=True, detailed=True)
print("FLOPs:", flops)
print("MACs:", macs)
print("Params:", params)
output = model(fake_input)
print(output.logits[:,:,:10])

[2024-05-18 14:54:08,367] [INFO] [profiler.py:1218:get_model_profile] Flops profiler warming-up...


[2024-05-18 14:54:09,958] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 1:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         1.12 B  
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       133.47 GMACs
fwd flops per GPU:                                                      267.23 G
fwd flops of model = fwd flops per GPU * mp_size:                     

# 测试 rms+rope一致性

In [3]:
from flash_attn.layers.rotary import apply_rotary_emb_func as __apply_rotary_emb_func
from flash_attn.ops.rms_norm import rms_norm as __rms_norm
from torch import nn
import torch
class Qwen2RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Qwen2RMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

In [12]:
eps = 1e-6
dim = 16
input  = torch.randn([1,dim],device="cuda:0",dtype=torch.bfloat16)
torch_rmsnorm = Qwen2RMSNorm(dim, eps=eps).to("cuda:0")
print(torch_rmsnorm.weight.dtype)
output1 = torch_rmsnorm(input)
#=
output2 = __rms_norm(input, torch_rmsnorm.weight, torch_rmsnorm.variance_epsilon)
print(output1)
print(output2)

torch.float32
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', grad_fn=<MulBackward0>)
tensor([[ 0.3965, -0.6992, -1.8359, -0.8242, -1.3672,  1.4688,  0.1836,  1.0703,
         -1.0859, -1.9062, -0.2969, -0.3633, -0.9648, -0.0815,  0.1748, -0.3105]],
       device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<DropoutAddLayerNormFnBackward>)


In [ ]:
# test recurrentgemma
from transformers import AutoConfig
from recurrentgemma.modeling_recurrent_gemma import RecurrentGemmaForCausalLM
config = AutoConfig.from_pretrained("./recurrentgemma",trust_remote_code=True)
model = RecurrentGemmaForCausalLM(config)
print(model)

# load model

In [2]:
import os
import sys
import torch
import numpy  as np
import lightning as L
from deepspeed.profiling.flops_profiler import get_model_profile  # type: ignore
# fabric = L.Fabric(devices=1, precision="bf16-mixed")
fabric = L.Fabric(accelerator="cpu")
current_dir = os.getcwd()
print(current_dir)
sys.path.append(os.path.join(current_dir, "steel_modify_from_qwen_1_5"))
from transformers import AutoConfig
from steel_modify_from_qwen_1_5.modeling_steel import SteelForCausalLM
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)
# config.mlp_type = "raw"
# config.FFN_type = "softmoe_v3"
ckpt = "/data/model/llm/lightning_model/step-1060000-iter-8480000-ckpt/state.pth"
model = SteelForCausalLM(config).to("cpu")
state = {"model": model}
fabric.load(ckpt, state)


/home/gujiangang/data_struct/Steel-LLM/pretrain_modify_from_TinyLlama/model
zhanshijin: use torch rmsnorm


{'optimizer': {'state': {'lm_head.weight': {'step': tensor(1060000.),
    'exp_avg': tensor([[ 1.0723e-05, -5.9666e-06, -9.3059e-06,  ..., -1.1473e-05,
             -4.9136e-06,  1.5291e-06],
            [ 2.4363e-06,  6.3743e-06, -1.3942e-05,  ...,  1.6499e-06,
              2.3013e-05, -6.1599e-07],
            [ 1.6299e-06, -4.0630e-06,  6.6359e-06,  ...,  9.3626e-07,
              1.9461e-05,  7.1244e-06],
            ...,
            [ 1.7267e-10,  7.8806e-11, -1.0519e-10,  ...,  1.2149e-10,
              4.5572e-10,  3.6193e-10],
            [ 1.7570e-10,  7.9999e-11, -1.1105e-10,  ...,  1.2108e-10,
              4.8826e-10,  3.6271e-10],
            [ 1.7501e-10,  7.5612e-11, -1.0016e-10,  ...,  1.2406e-10,
              4.5152e-10,  3.6147e-10]]),
    'exp_avg_sq': tensor([[1.7326e-09, 4.1006e-09, 1.3978e-09,  ..., 2.7938e-09, 2.6405e-08,
             1.7322e-09],
            [1.3940e-09, 1.8666e-09, 1.6805e-09,  ..., 7.3562e-10, 1.0232e-08,
             1.4314e-09],
          

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./tokenizer_from_qwen_moe_chat")
print(tokenizer.pad_token_id)
string = '''重力加速度是'''
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": string}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# string = "python怎么写for循环"
model_inputs = tokenizer([text], return_tensors="pt").to("cpu")
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=20,
    repetition_penalty = 1.1,
    temperature = 0.01,
)
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


151643
system
You are a helpful assistant.
user
重力加速度是
assistant
10m/s2, 5m/s2, 3m/s2, 2


保存模型

In [4]:
model.save_pretrained("/data/model/llm/hf_model/steel-llm-step-1060000-ckpt")

hf加载模型

In [1]:

# model_dir = "/data/model/llm/hf_model/steel-llm-step-1060000-ckpt"
model_dir = "/data/model/llm/fintuned_model/steel_106k_infinity_all/checkpoint-3906"
import sys
sys.path.append(model_dir)
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
        model_dir, trust_remote_code=True
    )
model = AutoModelForCausalLM.from_pretrained(
        model_dir, device_map="auto", trust_remote_code=True
    ).eval()


/root/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
mlp_class: SteelSENet
FFN: SteelSoftMoeV3
zhanshijin: now use _attn_implementation is sdpa, you can choose from dict_keys(['eager', 'flash_attention_2', 'sdpa'])


zhanshijin: surrport flash_attn_2
zhanshijin: if flash attn surrport window:True
zhanshijin: use torch rmsnorm


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./tokenizer_from_qwen_moe_chat")
print(tokenizer.pad_token_id)
string = '''下列设备属于资源子网的是哪个,请直接给出选项，不要做解释: \nA. 计算机软件\nB. 网桥\nC. 交换机\nD. 路由器\n'''
# string = """在OSI参考模型中，直接为会话层提供服务的是，请直接给出选项:\nA. 应用层\nB. 表示层\nC. 传输层\nD. 网络层"""
#string = """A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around US\\$90, its Japanese inventors said Tuesday.\n\nWhat best summarizes the content of the above article?\npick from the following. (I). World. (II). Sports. (III). Business. (IV). Science/Tech."""
#string = """你是谁"""
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": string}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# string = "python怎么写for循环"
model_inputs = tokenizer([text], return_tensors="pt").to("cuda:7")
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=200,
    repetition_penalty = 1.1,
    temperature = 0.01,
)
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


151643
system
You are a helpful assistant.
user
下列设备属于资源子网的是哪个,请直接给出选项，不要做解释: 
A. 计算机软件
B. 网桥
C. 交换机
D. 路由器

assistant
计算机网络是指通过通信介质将不同地理位置的计算机连接起来形成一个整体。它包括局域网和广域网两种形式。
广域网是利用互联网技术建立的一种公共网络，其主要特点是资源共享、信息传递速度快、传输距离远等。而计算机网络则是指在计算机之间实现数据、语音、图像等多种信息交互的技术系统。
因此，答案为：D


In [ ]:
model.push_to_hub("gqszhanshijin/Steel-LLM")

# softmoe v1

In [3]:
from torch import nn
import torch.nn.functional as F
import torch
from transformers import AutoConfig
config = AutoConfig.from_pretrained("./steel_modify_from_qwen_1_5",trust_remote_code=True)

class SteelSoftMoeV1(nn.Module):
    def __init__(self, config, layer=None):
        super().__init__()
        self.config = config
        # self.experts = nn.ModuleList([layer(config) for _ in range(config.n_experts)])
        self.experts = nn.ModuleList([nn.Linear(config.hidden_size, 10) for _ in range(config.n_experts)])
        self.gating = nn.Linear(config.hidden_size, config.n_experts)
    def forward(self, x):
        weights = self.gating(x)
        weights = nn.functional.softmax(weights, dim=-1, dtype=torch.float32).to(x.dtype)
        outputs = torch.stack( 
            [expert(x) for expert in self.experts], dim=2) 
        weights = weights.unsqueeze(-1)
        return torch.sum(outputs * weights, dim=2)
    
fake_input = torch.randn([1,30,config.hidden_size])
layer = SteelSoftMoeV1(config)
print(layer(fake_input).shape)

torch.Size([1, 30, 10])


In [4]:
import torch
a = torch.tensor([1.0,2.0], device="cuda:0",dtype=torch.bfloat16)
b = torch.tensor([1.0,2.0], device="cuda:0",dtype=torch.float32)
c = a- b
print(c.dtype)

torch.float32
